# 使用私有数据微调ChatGLM3 #

In [1]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [9]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'
train_data_path = 'data/zhouyi_dataset_20250618_214633.csv'
eval_data_path = None
seed = 8
max_input_length = 512
max_output_length = 1536
lora_rank = 16
lora_alpha = 32
lora_dropout = 0.05
prompt_text = ''

## 数据处理

In [10]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset) #dataset是字典


DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


In [11]:
dataset['train'].features # features 是数据集特征描述，包含每个列的名称和类型

{'content': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [12]:
dataset['train']['content'] # content是键

['蒙卦代表什么？',
 '周易中的蒙卦含义是什么？',
 '请解释一下蒙卦。',
 '蒙卦在周易中是什么象征？',
 '周易蒙卦的深层含义是什么？',
 '蒙卦和教育启蒙有什么联系？',
 '周易的蒙卦讲述了什么？',
 '蒙卦是怎样的一个卦象？',
 '蒙卦在周易中怎样表达教育的概念？',
 '蒙卦的基本意义是什么？',
 '周易中蒙卦的解释是什么？',
 '蒙卦在周易中代表了哪些方面？',
 '蒙卦涉及哪些哲学思想？',
 '周易中蒙卦的象征意义是什么？',
 '蒙卦的主要讲述内容是什么？',
 '周易蒙卦的核心思想是什么？',
 '蒙卦和启蒙教育之间有何联系？',
 '在周易中，蒙卦象征着什么？',
 '请描述蒙卦的含义。',
 '蒙卦在周易哲学中扮演什么角色？',
 '屯卦代表什么？',
 '周易中的屯卦含义是什么？',
 '请解释一下屯卦。',
 '屯卦在周易中是什么象征？',
 '周易屯卦的深层含义是什么？',
 '屯卦和教育启蒙有什么联系？',
 '周易的屯卦讲述了什么？',
 '屯卦是怎样的一个卦象？',
 '屯卦在周易中怎样表达教育的概念？',
 '屯卦的基本意义是什么？',
 '周易中屯卦的解释是什么？',
 '屯卦在周易中代表了哪些方面？',
 '屯卦涉及哪些哲学思想？',
 '周易中屯卦的象征意义是什么？',
 '屯卦的主要讲述内容是什么？',
 '周易屯卦的核心思想是什么？',
 '屯卦和启蒙教育之间有何联系？',
 '在周易中，屯卦象征着什么？',
 '请描述屯卦的含义。',
 '屯卦在周易哲学中扮演什么角色？',
 '需卦代表什么？',
 '周易中的需卦含义是什么？',
 '请解释一下需卦。',
 '需卦在周易中是什么象征？',
 '周易需卦的深层含义是什么？',
 '需卦和教育启蒙有什么联系？',
 '周易的需卦讲述了什么？',
 '需卦是怎样的一个卦象？',
 '需卦在周易中怎样表达教育的概念？',
 '需卦的基本意义是什么？',
 '周易中需卦的解释是什么？',
 '需卦在周易中代表了哪些方面？',
 '需卦涉及哪些哲学思想？',
 '周易中需卦的象征意义是什么？',
 '需卦的主要讲述内容是什么？',
 '周易需卦的核心思想是什么？',
 '需卦和启蒙教育之间有何联系？',
 '在周易中，需卦象征着什么？',
 '请描述需卦的

In [13]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dateset."
    
    # 生成一组随机数 picks 用作 dataset索引
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    # 生成 df表格
    df = pd.DataFrame(dataset[picks]) # 返回picks列表的行索引写到表格df
    for column,typ in dataset.features.items():# 固定格式目的是转换数据集中的分类标签和分类标签序列
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
        
    

In [14]:
show_random_elements(dataset["train"],num_examples=5)

,content,summary
0,蒙卦的基本意义是什么？,蒙卦在周易中象征着启蒙与迷惑。此卦由下卦坎（水）和上卦艮（山）组成，卦象为山下有泉，象征泉水喷涌而出，但仍有险阻。这一卦象寓意着蒙昧和回还往复，表明智慧未开，蒙昧闭塞，犹豫不决，缺乏果断。在卦辞中提到‘匪我求童蒙，童蒙求我’，表明不是主动去寻求启蒙，而是蒙昧的人主动寻求教化。初筮告，再三渎，渎则不告，意味着初次占筮可以得到启示，但若轻慢不敬地反复占问，则不会得到启示。蒙卦的核心哲学是强调启蒙的重要性，君子应以果敢坚毅的行动来培养自身的品德。在事业上，蒙卦暗示开始时可能混乱无序，但通过勇敢坚毅的行动和接受严格教育，可以扭转局面。在经商和求名方面，蒙卦建议小心谨慎，不得急功近利，应树立高尚的商业道德和接受良好的基础教育。在婚恋方面，蒙卦提醒注意考察对方品德，夫妻需相互宽容、理解。总之，蒙卦教导人们在迷惘困顿中寻求教化，通过果敢坚毅的行动和接受教育，最终达到启蒙和通达的状态。
1,讼卦和教育启蒙有什么联系？,讼卦在周易中代表争讼、诉讼的情境。此卦由上卦乾（天）和下卦坎（水）组成，象征天水相违，流向相背，事理乖舛，从而引发争讼。卦辞指出，虽然有利可图，但需警惕戒惧，中期吉利，但最终可能转为凶险。此卦有利于遇见有德行的大人物，但不利于涉越大河。象辞解释为君子应从卦象中汲取教训，在做事谋事之初就要谨慎小心。《断易天机》认为此卦为离宫游魂卦，上乾刚强，下坎险陷，刚险相争必然导致争讼，多有不吉。邵雍解为天高水深，达远不亲，应慎谋退守，敬畏无凶。傅佩荣认为此卦表示时运功名受阻，财运开始谨慎终可获利，家宅宜求淑女，身体需预防胜于治疗。传统解卦指出此卦为异卦相叠，乾为刚健，坎为险陷，刚与险对立，必然产生争讼。运势上事与愿违，宜防陷阱；事业上起初顺利，但继而受挫，需警惕谨慎，避免固执己见，退而让人以求化解；经商上和气生财，吃亏是福，应坚持公正公平互利原则；求名上不利，应坚守纯正，隐忍自励；婚恋上虽不尽人意，但双方理解可处，应以温和方式处理生活；决策上应避免争强好胜，知足适可而止，接受教训，引以为戒。
2,需卦和启蒙教育之间有何联系？,需卦在周易中象征着等待与蓄势。此卦由下卦乾（天）和上卦坎（水）组成，代表云升于天，等待雨水降临的意象。卦辞‘有孚，光亨，贞吉。利涉大川’表明，坚守诚信，亨通顺利，问卦必吉，且利于渡过大河。象传‘云上于天，需；君子以饮食宴乐’说明，君子应如云在天，耐心等待，并在等待中保持宴饮安乐的生活态度。《断易天机》指出需卦为坤宫游魂卦，意味着等待时机成熟，虽前有险阻，但耐心涉过大川则利。邵雍认为需卦象征坎陷当前，应等待时机，急进则凶。傅佩荣则强调时机未成熟时需耐心等待，适用于时机、财运、身体等方面。传统解卦认为需卦下乾上坎，刚逢险阻，宜稳健等待，不可冒进。大象解法以云在天上未雨象喻等待时机，运势上强调待时而行，事业、经商、求名、婚恋等方面均需耐心等待，积累实力，待时机成熟则可成功。决策上建议积蓄力量，谨慎行动，坦然面对困难，时机成熟后则一帆风顺。
3,需卦涉及哪些哲学思想？,需卦在周易中象征着等待与蓄势。此卦由下卦乾（天）和上卦坎（水）组成，代表云升于天，等待雨水降临的意象。卦辞‘有孚，光亨，贞吉。利涉大川’表明，坚守诚信，亨通顺利，问卦必吉，且利于渡过大河。象传‘云上于天，需；君子以饮食宴乐’说明，君子应如云在天，耐心等待，并在等待中保持宴饮安乐的生活态度。《断易天机》指出需卦为坤宫游魂卦，意味着等待时机成熟，虽前有险阻，但耐心涉过大川则利。邵雍认为需卦象征坎陷当前，应等待时机，急进则凶。傅佩荣则强调时机未成熟时需耐心等待，适用于时机、财运、身体等方面。传统解卦认为需卦下乾上坎，刚逢险阻，宜稳健等待，不可冒进。大象解法以云在天上未雨象喻等待时机，运势上强调待时而行，事业、经商、求名、婚恋等方面均需耐心等待，积累实力，待时机成熟则可成功。决策上建议积蓄力量，谨慎行动，坦然面对困难，时机成熟后则一帆风顺。
4,蒙卦涉及哪些哲学思想？,蒙卦在周易中象征着启蒙与迷惑。此卦由下卦坎（水）和上卦艮（山）组成，卦象为山下有泉，象征泉水喷涌而出，但仍有险阻。这一卦象寓意着蒙昧和回还往复，表明智慧未开，蒙昧闭塞，犹豫不决，缺乏果断。在卦辞中提到‘匪我求童蒙，童蒙求我’，表明不是主动去寻求启蒙，而是蒙昧的人主动寻求教化。初筮告，再三渎，渎则不告，意味着初次占筮可以得到启示，但若轻慢不敬地反复占问，则不会得到启示。蒙卦的核心哲学是强调启蒙的重要性，君子应以果敢坚毅的行动来培养自身的品德。在事业上，蒙卦暗示开始时可能混乱无序，但通过勇敢坚毅的行动和接受严格教育，可以扭转局面。在经商和求名方面，蒙卦建议小心谨慎，不得急功近利，应树立高尚的商业道德和接受良好的基础教育。在婚恋方面，蒙卦提醒注意考察对方品德，夫妻需相互宽容、理解。总之，蒙卦教导人们在迷惘困顿中寻求教化，通过果敢坚毅的行动和接受教育，最终达到启蒙和通达的状态。


In [15]:
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

/root/miniconda3/envs/chatglmdemo/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [ ]:
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """
    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input',None) and example['input'].strip():
        question += f'\n{example['input']}'
        
    # 构建答案文本
    answer = example['summary']
    
    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)
    
    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:
        q_ids = q_ids[:max_input_length - 2] # 保留空间给gmask和bos标记
    if len(a_ids) > max_output_length - 1:
        a_ids = a_ids[:max_output_length - 1] # 保留空间给eos标记
        
    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2
    
    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]
    
    return {'input_ids':input_ids,'labels':labels}

In [ ]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example:tokenize_func(example, tokenizer),
    batched=False,
    remove_column=column_names
)

In [ ]:
tokenized_dataset = tokenized_dateset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()